# Data Science en Entreprise

## API pôle emploi

Autorisation d’accès à l’api des offres d’emplois

Compte pole emploi :    
username : mohammed.didi@outlook.com    
password : Projet_Data1



In [37]:
import requests


def obtenir_token(client_id, client_secret):
    """
    Cette fonction obtient un token d'accès pour l'API de Pôle Emploi.
    """
    url_token = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire'
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': 'api_offresdemploiv2 o2dsoffre'
    }
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    response = requests.post(url_token, data=auth_data, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Erreur lors de la requête : {response.text}")
    else:
        data = response.json()
        return data['access_token']
    
# Utilisation de la fonction
client_id = 'PAR_projetdata_29c785675807663802eabdf57858fc4e2ecd4eba4de8d05eee293bfdcacefae5'
client_secret = '8a48c4726bb2d33cafb7a636703a3b0ad70e4098b525063d2a99387a95a76b70'


In [43]:
import requests
from datetime import datetime
import pandas as pd
import json


def rechercher_offres(token, mot_cle, mois, annee, nb_offres=10):
    """
    Recherche les offres d'emploi dans le domaine des RH pour un mois et une année spécifiques.
    """
    url_offres = 'https://api.emploi-store.fr/partenaire/offresdemploi/v2/offres/search'
    headers = {'Authorization': f'Bearer {token}'}

    # Calculer les dates de début et de fin du mois de novembre
    debut_mois = f"{annee}-{mois:02d}-01T00:00:00Z"
    fin_mois = f"{annee}-{mois:02d}-30T00:00:00Z"  # Novembre a 30 jours

    params = {
        'motsCles': mot_cle,  # Mots-clés pour le domaine des Ressources Humaines
        'range': f'0-{nb_offres-1}',  # Les 10 premières offres
        'minCreationDate': debut_mois,
        'maxCreationDate': fin_mois
    }

    response = requests.get(url_offres, headers=headers, params=params)

    try:
        return response.json()
    except:
        raise Exception(f"Erreur lors de la requête : {response.text}")



token = obtenir_token(client_id, client_secret)
offres_Data_novembre = rechercher_offres(token, 'Data', 11, 2023)  # Exemple pour novembre 2023
# json to dataframe
# jsoon to dataframe
df = pd.DataFrame.from_dict(offres_Data_novembre['resultats'])


In [44]:
df

,id,intitule,description,dateCreation,dateActualisation,lieuTravail,romeCode,romeLibelle,appellationlibelle,entreprise,...,secteurActiviteLibelle,qualitesProfessionnelles,origineOffre,offresManqueCandidats,experienceCommentaire,formations,deplacementCode,deplacementLibelle,langues,conditionExercice
0,165PPDB,Data Engineer Développement Oracle ODI F/H (H/F),Profil et prérequis :\nVous êtes inscrit(e) à ...,2023-11-29T16:27:49.000Z,2023-12-01T11:33:34.000Z,"{'libelle': '92 - PUTEAUX', 'latitude': 48.884...",M1805,Études et développement informatique,Développeur / Développeuse Big Data,{'entrepriseAdaptee': False},...,Conseil en systèmes et logiciels informatiques,"[{'libelle': 'Avoir le sens du service', 'desc...","{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN,NaN,NaN,NaN
1,165PMPF,TECHNICIEN MASTER DATA (H/F),Vous aurez pour mission : \n- D'assurer la ges...,2023-11-29T16:02:46.000Z,2023-11-30T09:21:12.000Z,"{'libelle': '27 - LE NEUBOURG', 'latitude': 49...",H1208,Intervention technique en études et conception...,Intégrateur / Intégratrice en informatique ind...,"{'nom': 'CRIT INTERIM', 'entrepriseAdaptee': F...",...,Activités des agences de travail temporaire,NaN,"{'origine': '1', 'urlOrigine': 'https://candid...",False,en Master Data ou diplôme,"[{'codeFormation': '31845', 'domaineLibelle': ...",NaN,NaN,NaN,NaN
2,165PGZH,Data Engineer - (H/F),Nous recrutons un/une Ingénieur Data Engineer ...,2023-11-29T14:36:53.000Z,2023-11-29T14:36:55.000Z,"{'libelle': '31 - TOULOUSE', 'latitude': 43.60...",M1403,Études et prospectives socio-économiques,Data scientist,"{'nom': 'CS GROUP - FRANCE', 'entrepriseAdapte...",...,Programmation informatique,[{'libelle': 'Prendre des initiatives et être ...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,[{'niveauLibelle': 'Bac+5 et plus ou équivalen...,1,Jamais,NaN,NaN
3,165NSBV,Data Steward - F/H (H/F),Description du poste\n\nDans le cadre de son o...,2023-11-29T10:08:48.000Z,2023-12-01T10:24:28.000Z,"{'libelle': '92 - LE PLESSIS ROBINSON', 'latit...",M1403,Études et prospectives socio-économiques,Data analyst,"{'nom': 'CS GROUP - FRANCE', 'description': 'C...",...,Conseil en systèmes et logiciels informatiques,NaN,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN,NaN,NaN,NaN
4,165NLBR,Ingénieur CVC - Activité DATA CENTER (H/F),Au sein de l'activité DATACENTER les missions ...,2023-11-28T19:01:26.000Z,2023-11-28T19:01:28.000Z,"{'libelle': '94 - IVRY SUR SEINE', 'latitude':...",F1106,Ingénierie et études du BTP,Ingénieur / Ingénieure bâtiment,"{'nom': 'PROJEX', 'entrepriseAdaptee': False}",...,"Ingénierie, études techniques",NaN,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN,NaN,NaN,NaN
5,165MZBN,Data scientist,La division « Database Administration & Data M...,2023-11-28T15:07:22.000Z,2023-12-01T14:28:55.000Z,{'libelle': 'Luxembourg (Frontalier)'},M1403,Études et prospectives socio-économiques,Data scientist,"{'nom': 'LUX ADVISORY', 'entrepriseAdaptee': F...",...,Conseil en systèmes et logiciels informatiques,NaN,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN,NaN,"[{'libelle': 'Anglais', 'exigence': 'E'}]",NaN
6,165MSWS,Ingénieure / Ingénieur DATA (H/F),Description du poste\nGarant du patrimoine de ...,2023-11-28T12:50:11.000Z,2023-11-29T11:14:27.000Z,"{'libelle': '38 - GRENOBLE', 'latitude': 45.18...",M1403,Études et prospectives socio-économiques,Data analyst,"{'nom': 'ARTELIA', 'entrepriseAdaptee': False}",...,"Ingénierie, études techniques",NaN,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN,NaN,NaN,NaN
7,165MSVZ,Data Project Manager F/H (H/F),Description du poste\nGarant du patrimoine de ...,2023-11-28T12:48:42.000Z,2023-11-29T10:49:00.000Z,"{'libelle': '38 - GRENOBLE', 'latitude': 45.18...",M1803,Direction des systèmes d'information,Project Manager,"{'nom': 'ARTELIA', 'entrepriseAdaptee': False}",...,"Ingénierie, études techniques",NaN,"{'origine': '1', 'urlOrigine': 'ht

In [45]:
df.columns

Index(['id', 'intitule', 'description', 'dateCreation', 'dateActualisation',
       'lieuTravail', 'romeCode', 'romeLibelle', 'appellationlibelle',
       'entreprise', 'typeContrat', 'typeContratLibelle', 'natureContrat',
       'experienceExige', 'experienceLibelle', 'competences', 'salaire',
       'dureeTravailLibelle', 'dureeTravailLibelleConverti', 'alternance',
       'contact', 'agence', 'nombrePostes', 'accessibleTH',
       'qualificationCode', 'qualificationLibelle', 'codeNAF',
       'secteurActivite', 'secteurActiviteLibelle', 'qualitesProfessionnelles',
       'origineOffre', 'offresManqueCandidats', 'experienceCommentaire',
       'formations', 'deplacementCode', 'deplacementLibelle', 'langues',
       'conditionExercice'],
      dtype='object')

In [46]:
# save to csv in ../data

df.to_csv('../data/offres_data_novembre.csv', index=False)
